# Domestic Violence DFJ

## Contents
#### Setup
1. [import_packages](#import_packages) 
2. [define_key_variables](#define_key_variables) 

## 1. Import packages and set options 
<a name="import_packages"></a>

In [1]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

## 2. Define key variables to be used throughout the notebook 
<a name="define_key_variables"></a>

In [2]:
#this is the database we will be extracting from
database = "familyman_dev_v3" 

#this extracts the latest snapshot from athena
#get_snapshot_date = f"SELECT mojap_snapshot_date from {database}.events order by mojap_snapshot_date desc limit 1"
#snapshot_date = str(pydb.read_sql_query(get_snapshot_date)['mojap_snapshot_date'].values[0])

#this extracts the August snapshot from athena
snapshot_date = '2022-08-04'

#this is the athena database we will be storing our tables in
fcsq_database = "fcsq"

#this is the s3 bucket we will be saving data to
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

#change these to the current quarter and year not the quarter being published
latest_quarter = 3
latest_year = 2022

## 3. Dom_violence_Orders table 

In [3]:
create_Dom_Violence_Orders = f"""
SELECT Year, Quarter, Event_court, count(*) as Disposals
FROM fcsq.DV_ORDS_FINAL
WHERE Year > 2010
GROUP BY Year, Quarter, Event_court
ORDER BY Year, Quarter, Event_court;
"""

pydb.create_temp_table(create_Dom_Violence_Orders,'Dom_Violence_Orders');

In [4]:
#check = "SELECT COUNT(*) as Count from __temp__.Dom_Violence_Orders"
#pydb.read_sql_query(check)

## 4. Dom_violence_Cases table 

In [5]:
create_Dom_Violence_Cases = f"""
SELECT Year, Quarter, Event_court, count(*) as Case_Count
FROM fcsq.DV_ORDS_FINAL
WHERE Year > 2010 and substring(case_number,5,1) = 'F'
GROUP BY Year, Quarter, Event_court
ORDER BY Year, Quarter, Event_court;
"""

pydb.create_temp_table(create_Dom_Violence_Cases,'Dom_Violence_Cases');

In [6]:
#check = "SELECT COUNT(*) as Count from __temp__.Dom_Violence_Cases"
#pydb.read_sql_query(check)

## 5. Dom_violence_Merge table 

In [7]:
create_Dom_Violence_Merge = f"""
Select t1.Year,
t1.Quarter,
t1.Event_court,
COALESCE(t1.Disposals, 0) as Disposals,
COALESCE(t2.Case_Count, 0) as Case_Count,
'Domestic Violence' as Category,
'End' as Stage
from __temp__.Dom_Violence_Orders t1
FULL OUTER JOIN
__temp__. Dom_Violence_Cases t2 
on t1.Year = t2.Year AND t1.Quarter = t2.Quarter AND t1.Event_court = t2.Event_court
WHERE NOT (t1.year = {latest_year} AND t1.quarter = {latest_quarter})

"""

pydb.create_temp_table(create_Dom_Violence_Merge,'Dom_Violence_Merge');

In [8]:
#check = "SELECT COUNT(*) as Count from __temp__.Dom_Violence_Merge"
#pydb.read_sql_query(check)

In [9]:
#check = "SELECT * from __temp__.Dom_Violence_Merge"
#pydb.read_sql_query(check)

## 6. Dom_violence_Format table 

In [10]:
create_Dom_Violence_Format = f"""
SELECT Category, 
Year, 
Quarter, 
Event_court as Court, 
Stage, 
Disposals as Count,
Case_Count as Cases
FROM __temp__.Dom_Violence_Merge;
"""

pydb.create_temp_table(create_Dom_Violence_Format,'Dom_Violence_Format');

In [11]:
#check = "SELECT COUNT(*) as Count from __temp__.Dom_Violence_Format"
#pydb.read_sql_query(check)

In [12]:
#check = "SELECT * from __temp__.Dom_Violence_Format"
#pydb.read_sql_query(check)